In [11]:
#import libraries

import pandas as pd
import numpy as np
import plotly.offline as pyo
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [12]:
#import datasets

train=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\train.csv')
test=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\test.csv')
meal_info=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\meal_info.csv')
fulfilment_center_info=pd.read_csv(r'C:\Users\hp\Downloads\demand prediction\fulfilment_center_info.csv')

In [13]:
#load test and train datasets
X_train=train.copy()
X_test=test.copy()
y_train=X_train['num_orders']
X_train=X_train.drop(['num_orders'],axis=1)



In [14]:
#prediction using xgb
import xgboost as xgb
model=xgb.XGBRegressor(objective="reg:linear", random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


[16:11:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
#create prediction dataframe y_pred
y_pred=pd.DataFrame(y_pred)
y_pred=list(y_pred[0])
week=X_test['week']
y_pred=list(zip(list(week),y_pred))
y_pred=pd.DataFrame(y_pred,columns=['week','num_orders'])
y_pred=y_pred.groupby('week',as_index=False)['num_orders'].sum()


In [16]:
#create historical dataframe X_hist
y_train=pd.DataFrame(y_train,columns=['num_orders'])
X_hist=list(zip(list(X_train['week']),list(y_train['num_orders'])))
X_hist=pd.DataFrame(X_hist,columns=['week','num_orders'])
X_hist=X_hist.groupby('week',as_index=False)['num_orders'].sum()



In [17]:
#plot the historical and predicted values

import plotly.graph_objects as go

fig=go.Figure()
fig.add_trace(go.Scatter(x=X_hist.week,y=X_hist.num_orders,name='Historical',mode='lines+markers',line=dict(color='firebrick',width=1)))
fig.update_layout(title='Total Number of Orders per Week',xaxis_title='Week',yaxis_title='Number of Orders')

fig.add_trace(go.Scatter(x=y_pred.week,y=y_pred.num_orders,name='Predicted',mode='lines+markers',line=dict(color='blue',width=1)))
fig.update_layout(title='Total Number of Orders per Week',xaxis_title='Week',yaxis_title='Number of Orders')

fig.show()